# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [1]:
text = open('2600-0.txt', 'r').read()[2:]
len(text)

3227579

In [2]:
import string
import re

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', ',—*)[-];:/@!“?’#%=”‘('))
    text = re.sub(r"\s+", ' ', text)
    
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    return text

text = preprocess_text(text)
# assert len(text) == 3141169
print(len(text))

3130292


In [4]:
# !"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~—’“”

In [3]:
text = text.split('.')
text = [x.strip() for x in text]

In [4]:
text = [x for x in text if x != '']

len(text)

26880

In [167]:
from collections import Counter, defaultdict
import nltk
from sklearn.base import TransformerMixin
import tokenize
from nltk import FreqDist
from nltk.util import ngrams
import operator

# stopwords = set(nltk.corpus.stopwords.words('english'))


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        new_text = ' '.join((text))
        self.itos = [x for x in list(text) if x != ' ']
#         self.stoi = {' '.join(token): count for token, count in Counter(text).items()}
        self.stoi = {x[1]: x[0] for x in enumerate(set(self.itos))}

        
        bigram_text = ngrams(text, 2)
        frequency = FreqDist(bigram_text)
                        
        while len(self.itos) < self.vocab_size:
            # count bigram freqencies in the text
            if len(frequency) == 0:
                break
            new_token = max(frequency.items(), key=operator.itemgetter(1))[0]
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id + 1
            
            del frequency[new_token]            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = ' '.join(text)
        text = ' '.join(list(text))
        
        for token, token_id in self.stoi.items():
            if isinstance(token, tuple):
                token = ' '.join(token)
                print(token)
            text = text.replace(str(token), str(token_id))
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        
        for key, num in self.stoi.items():
            if num == int(tok.strip()):
                return key
        return ' '
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text.split()))
                        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text[:3] + text[4:10])

In [168]:
assert bpe.decode(tokenized_text[:2]) == text[0][0]

In [169]:
bpe.decode(tokenized_text[:9])

't5 '

In [ ]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = np.zeros()
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = # TODO
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        result = # TODO approximate probability by counters
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """            
        self.proba = # TODO count 3-grams in the text
        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [ ]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    
    beam = # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if # TODO process terminal token
            else:    
                proba = # probability vector of the next token
                best_k = # top-k most probable tokens
                # TODO update candidates' sequences and corresponding probabilities
                
        beam = # select top-k most probable sequences from candidates
    return beam
    

In [ ]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
    

In [ ]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [ ]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities

In [ ]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [ ]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)